In [14]:
import pandas as pd
from datetime import datetime, timedelta

def transform_employee_data():
    """
    Transforms employee data from 'input.csv' to historical format in 'output.csv'.
    """

    df = pd.read_csv('input.csv')
    df.fillna(method='ffill', inplace=True)  
    df.replace(r'^\s*$', float('nan'), regex=True, inplace=True) 

    output_rows = []
    for _, row in df.iterrows():
        base_data = dict(row)

        # Process compensations
        for i in range(1, 3):
            comp = base_data[f'Compensation {i}']  # Get the compensation value
            comp_date_str = base_data[f'Compensation {i} date']
            if pd.notnull(comp_date_str):
                # Try parsing in both formats
                try: 
                    comp_date = datetime.strptime(comp_date_str, '%d.%m.%y')
                except ValueError:
                    comp_date = datetime.strptime(comp_date_str, '%Y-%m-%d')

                output_rows.append({
                    **base_data,
                    'Compensation': comp,
                    'Last Compensation': comp,
                    'Last Pay Raise Date': comp_date, 
                    'Variable Pay': 0,  
                    'Tenure in Org': comp_date, 
                    'Effective Date': comp_date,
                    'End Date': (comp_date + timedelta(days=-1)).replace(year=2100) if i == 2 else comp_date + timedelta(days=-1)
                })

        # Process reviews (similar logic as compensations)
        for i in range(1, 3):
            review = base_data[f'Review {i}']
            review_date_str = base_data[f'Review {i} date']
            if pd.notnull(review_date_str):
                try: 
                    review_date = datetime.strptime(review_date_str, '%d.%m.%y')
                except ValueError:
                    review_date = datetime.strptime(review_date_str, '%Y-%m-%d')
                output_rows.append({
                    **base_data,
                    'Performance Rating': review,
                    'Effective Date': review_date,
                    'End Date': (review_date + timedelta(days=-1)).replace(year=2100) if i == 2 else review_date + timedelta(days=-1)
                })

        # Process engagements (similar logic) 
        for i in range(1, 3):
            engagement_score = base_data[f'Engagement {i}']
            engagement_date_str = base_data[f'Engagement {i} date']
            if pd.notnull(engagement_date_str):
                try:
                    engagement_date = datetime.strptime(engagement_date_str, '%d.%m.%y')
                except ValueError:
                    engagement_date = datetime.strptime(engagement_date_str, '%Y-%m-%d')
                output_rows.append({
                    **base_data,
                    'Engagement Score': engagement_score,
                    'Effective Date': engagement_date,
                    'End Date': (engagement_date + timedelta(days=-1)).replace(year=2100) if i == 2 else engagement_date + timedelta(days=-1)
                })


    output_df = pd.DataFrame(output_rows)
    output_df.to_csv('output.csv', index=False)

transform_employee_data()
